# Digit Classifier using CNNs

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Loading the dataset using TensorFlow API:

In [ ]:
mnist = tf.keras.datasets.mnist
(training_data, training_labels), (validation_data, validation_labels) = mnist.load_data()

Preprocessing data:

In [3]:
training_data = training_data.reshape(-1, 28, 28, 1)
validation_data = validation_data.reshape(-1, 28, 28, 1)

training_datagen = ImageDataGenerator(
    rescale = 1 / 255,
    rotation_range = 30,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    zoom_range = 0.1,
    shear_range = 0.1,
    fill_mode = 'constant'
)

validation_datagen = ImageDataGenerator(
    rescale = 1 / 255
)

training_generator = training_datagen.flow(
    training_data,
    training_labels
)

validation_generator = validation_datagen.flow(
    validation_data,
    validation_labels,
    shuffle = False
)

Model:

In [21]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (28, 28, 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (5, 5), activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(96, (5, 5), activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (5, 5), activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(160, (5, 5), activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(192, (5, 5), activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation = 'softmax')
])

In [ ]:
model.summary()

In [ ]:
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 0.002 * 0.95 ** (epoch / 8)
)
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = tf.keras.optimizers.SGD(learning_rate = 0.002),
              metrics = ['accuracy']
)
history = model.fit(training_generator,
                    epochs = 200,
                    validation_data = validation_generator,
                    callbacks = [lr_schedule]
)

In [ ]:
train_acc = history.history['accuracy']
train_loss = history.history['loss']
validation_acc = history.history['val_accuracy']
validation_loss = history.history['val_loss']

num_epochs = len(train_acc)
epochs = range(num_epochs)

plt.plot(epochs, train_acc)
plt.plot(epochs, validation_acc)
plt.title("Training and Validation Accuracy")
plt.axis([0, num_epochs, 0.95, 1])
plt.figure()

plt.plot(epochs, train_loss)
plt.plot(epochs, validation_loss)
plt.title("Training and Validation Loss")
plt.axis([0, num_epochs, 0, 0.15])

In [8]:
model.save("CNN_MNIST_V4_5.h5")

Error Analysis

In [9]:
predictions = model.predict(validation_generator)
predictions = np.argmax(predictions, axis = 1)
len(predictions)
incorrect = []
for i in range(10000):
  if predictions[i] != validation_labels[i]:
    incorrect.append(i)

In [ ]:
i = incorrect[0]
print(f"Expected: {validation_labels[i]}, Predicted: {predictions[i]}")
plt.imshow(validation_data[i].reshape(28, 28), cmap = 'gray')
plt.figure()